# Dispatch for `AbstractBond` objects

Since a lot of information about bond objects is already encoded into their type signature, this signature can be used in dispatch to greatly reduce the number of `if` statements as well as greatly improve the performance of your code.

This tutorial aims at providing an overview how to use dispatch for `AbstractBond` objects as well as giving some examples.

In [ ]:
using LatPhysBase

### Reminder: type structure for `AbstractBond`

Bond objects are in general subtype of `AbstractBond{L,N}` where the parametric types `L` and `N` give the **label type** as well as the **Bravais lattice dimension** of the surrounding unitcell / lattice respectively.

E.g. an object that is subtype of `AbstractBond{String,2}` would describe a bond with `String` labels in a Bravais lattice with 2 lattice vectors.



### How to use dispatch in function headers

If one wants to dispatch on a certain type signature, the first step is to include the types into the function header. One can do so by the following syntax:
```
function myfunction(b::B) where {B}
    # do something
end
```
In this way, function `myfunction` accepts any argument, since `B` is (implictly) subtype of `Any`.
To specialise to `AbstractBond` objects, one can use
```
function myfunction(b::B) where {B<:AbstractBond}
    # do something
end
```
which would only allow `b` to be of a subtype of `AbstractBond`, i.e. to be a bond.
Further specialization can be done by explicitly introducing the parametric types of `AbstractBond`.
```
function myfunction(b::B) where {L,N, B<:AbstractBond{L,N}}
    # do something
end
```
as these parametric types can be adjusted to the individual case as shown in the examples below.




# Examples

Let's discuss some applications in the following



### Example 1 - Unitcell / Lattice offset

In many cases, one is interested in the real space vector that is described by a certain bond object. For this, one not only needs the site coordinates of origin and destination site, but also the Bravais lattice vectors.

Leaving aside the availability of this information for the bond object (it should be provided within a unitcell or lattice object), one could ask the simple question: Given some lattice vectors and a bond, can we calculate the pure unitcell / lattice offset (where site positions are not relevant yet)? In formulas, this would be $\sum_{i=1}^N w_i \vec{a}_i$ where $w_i$ describes an element of the wrap.

Of course, we could write down this sum explicitly by looping over all lattice vectors etc., but we can also find explicit and faster syntax using dispatch.

In [ ]:
# N=0 case
function offset(b::B, ai::Vector{<:Vector{<:Real}}) where {L,B<:AbstractBond{L,0}}
     return zeros(3) # number is arbitrary here because we dont know which spatial dimension the sites have
end

# N=1 case
function offset(b::B, ai::Vector{<:Vector{<:Real}}) where {L,B<:AbstractBond{L,1}}
     return ai[1].*wrap(b)[1]
end

# N=2 case
function offset(b::B, ai::Vector{<:Vector{<:Real}}) where {L,B<:AbstractBond{L,2}}
     return ai[1].*wrap(b)[1] .+ ai[2].*wrap(b)[2]
end

# N=3 case
function offset(b::B, ai::Vector{<:Vector{<:Real}}) where {L,B<:AbstractBond{L,3}}
     return ai[1].*wrap(b)[1] .+ ai[2].*wrap(b)[2] .+ ai[3].*wrap(b)[3]
end

In [ ]:
# try out N=0
b = newBond( Bond{String,0}, 24,123, "mybond", NTuple{0,Int64}())
lv = Vector{Float64}[]
println( offset(b, lv) )

# try out N=1
b = newBond( Bond{String,1}, 24,123, "mybond", (-2,))
lv = Vector{Float64}[Float64[1,0,0]]
println( offset(b, lv) )

# try out N=2
b = newBond( Bond{String,2}, 24,123, "mybond", (-2,1))
lv = Vector{Float64}[Float64[1,0,0], Float64[0,1,0]]
println( offset(b, lv) )

# try out N=3
b = newBond( Bond{String,3}, 24,123, "mybond", (-2,-1,0))
lv = Vector{Float64}[Float64[1,0,0], Float64[0,1,0], Float64[0,-1,1]]
println( offset(b, lv) )

Note that a variation of the above example is implemented into the interface of `AbstractUnitcell` and `AbstractLattice` under the function name `vector` which needs a `AbstractBond` and an `AbstractUnitcell` or `AbstractLattice` object. See the unitcell or lattice type tutorial for more information.

### Example 2 - Plotting (dispatch by label)

Similar to how dispatch in plotting works for site objects (see [this](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/site_dispatch.ipynb) tutorial), one can also dispatch on bond label s.t. different labels have different implications in plotting.